In [1]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error, classification_report
import warnings
warnings.filterwarnings('ignore')
PATH_REPORTS = '../reports/'
lstm_results_path = 'test_results/LSTM_with_Attention_{asset}_test_results.csv'
mlp_results_path =  'test_results/MLP_{asset}_test_results.csv'

In [2]:
ASSETS = ["PETR3.SA", "PRIO3.SA", "VALE3.SA", "GGBR3.SA", "ABCB4.SA", "ITUB3.SA", "FLRY3.SA", "RADL3.SA"]
FEATURES_COLS = ['diff_close_mean_z_score']
LABEL_COL = ['meta']
SEQ_LEN = 60

In [3]:
def regression_metrics(y_test, y_pred):
    metrics = {
        'Mean Absolute Error (MAE)': mean_absolute_error(y_test, y_pred),
        'Mean Squared Error (MSE)': mean_squared_error(y_test, y_pred),
        'Root Mean Squared Error (RMSE)': mean_squared_error(y_test, y_pred, squared=False),
        'R-squared (R2)': r2_score(y_test, y_pred),
        'Mean Absolute Percentage Error (MAPE)': mean_absolute_percentage_error(y_test, y_pred)
    }
    
    metrics_df = pd.DataFrame(metrics, index=[0])
    return metrics_df

In [5]:
for asset in ASSETS:
    
    print(f"""###########################
# {asset}
###########################""")
    
    lstm_results = pd.read_csv(PATH_REPORTS + f'test_results/LSTM_with_Attention_{asset.replace(".", "_")}_features={"_".join(FEATURES_COLS)}__label={"_".join(LABEL_COL)}__sql_len={SEQ_LEN}_test_results.csv')
    
    reg_metrics_lstm = regression_metrics(lstm_results.y_test, lstm_results.y_pred)
    
    print("classification_report truc")
    print(classification_report([int(i) for i in lstm_results.y_test], [int(i) for i in lstm_results.y_pred]))
    
    print("classification_report round")
    print(classification_report([round(i) for i in lstm_results.y_test], [round(i) for i in lstm_results.y_pred]))

    print("Metricas regressão")
    display(reg_metrics_lstm)
    
    mlp_results = pd.read_csv(PATH_REPORTS + f'test_results/MLP_{asset.replace(".", "_")}_features={"_".join(FEATURES_COLS)}__label={"_".join(LABEL_COL)}__sql_len={SEQ_LEN}_test_results.csv')

    reg_metrics_mlp = regression_metrics(mlp_results.y_test, mlp_results.y_pred)

    print("classification_report truc")
    print(classification_report([int(i) for i in mlp_results.y_test], [int(i) for i in mlp_results.y_pred]))
    
    print("classification_report round")
    print(classification_report([round(i) for i in mlp_results.y_test], [round(i) for i in mlp_results.y_pred]))

    print("Metricas regressão")
    display(reg_metrics_mlp)

###########################
# PETR3.SA
###########################
classification_report truc
              precision    recall  f1-score   support

          -2       0.00      0.00      0.00        37
          -1       0.67      0.50      0.57       169
           0       0.67      0.95      0.78       458
           1       0.67      0.52      0.59       235
           2       0.75      0.10      0.17        62
           3       0.00      0.00      0.00         3

    accuracy                           0.67       964
   macro avg       0.46      0.34      0.35       964
weighted avg       0.65      0.67      0.63       964

classification_report round
              precision    recall  f1-score   support

          -3       0.00      0.00      0.00         2
          -2       0.87      0.12      0.22       104
          -1       0.60      0.75      0.67       216
           0       0.60      0.83      0.70       222
           1       0.58      0.72      0.64       260
          

,Mean Absolute Error (MAE),Mean Squared Error (MSE),Root Mean Squared Error (RMSE),R-squared (R2),Mean Absolute Percentage Error (MAPE)
0,0.450754,0.382812,0.618718,0.762755,0.719628


classification_report truc
              precision    recall  f1-score   support

          -2       1.00      0.03      0.05        37
          -1       0.67      0.55      0.60       169
           0       0.75      0.90      0.82       458
           1       0.71      0.76      0.74       235
           2       0.76      0.26      0.39        62
           3       0.00      0.00      0.00         3

    accuracy                           0.73       964
   macro avg       0.65      0.42      0.43       964
weighted avg       0.73      0.73      0.70       964

classification_report round
              precision    recall  f1-score   support

          -3       0.00      0.00      0.00         2
          -2       0.86      0.31      0.45       104
          -1       0.61      0.77      0.68       216
           0       0.66      0.71      0.69       222
           1       0.63      0.78      0.70       260
           2       0.60      0.41      0.48       137
           3       1.00

,Mean Absolute Error (MAE),Mean Squared Error (MSE),Root Mean Squared Error (RMSE),R-squared (R2),Mean Absolute Percentage Error (MAPE)
0,0.408648,0.354396,0.595312,0.780365,0.920992


###########################
# PRIO3.SA
###########################
classification_report truc
              precision    recall  f1-score   support

          -2       0.00      0.00      0.00        37
          -1       0.60      0.44      0.51       147
           0       0.66      0.93      0.77       455
           1       0.62      0.54      0.58       249
           2       1.00      0.03      0.05        76

    accuracy                           0.65       964
   macro avg       0.58      0.39      0.38       964
weighted avg       0.64      0.65      0.60       964

classification_report round
              precision    recall  f1-score   support

          -3       0.00      0.00      0.00        12
          -2       0.62      0.19      0.29        78
          -1       0.62      0.69      0.65       213
           0       0.58      0.79      0.67       231
           1       0.54      0.74      0.62       240
           2       0.73      0.26      0.38       170
          

...

,Mean Absolute Error (MAE),Mean Squared Error (MSE),Root Mean Squared Error (RMSE),R-squared (R2),Mean Absolute Percentage Error (MAPE)
0,0.442407,0.339467,0.582638,0.783842,76.145636


###########################
# RADL3.SA
###########################
classification_report truc
              precision    recall  f1-score   support

          -2       0.00      0.00      0.00        48
          -1       0.58      0.35      0.44       173
           0       0.67      0.91      0.77       453
           1       0.75      0.67      0.71       260
           2       0.50      0.07      0.12        28
           3       0.00      0.00      0.00         2

    accuracy                           0.68       964
   macro avg       0.42      0.34      0.34       964
weighted avg       0.63      0.68      0.64       964

classification_report round
              precision    recall  f1-score   support

          -3       0.00      0.00      0.00         9
          -2       0.53      0.08      0.14       110
          -1       0.54      0.60      0.57       219
           0       0.46      0.64      0.53       181
           1       0.60      0.84      0.70       287
          

,Mean Absolute Error (MAE),Mean Squared Error (MSE),Root Mean Squared Error (RMSE),R-squared (R2),Mean Absolute Percentage Error (MAPE)
0,0.446683,0.348089,0.58999,0.783336,2.314377


classification_report truc
              precision    recall  f1-score   support

          -2       0.82      0.19      0.31        48
          -1       0.65      0.68      0.66       173
           0       0.73      0.86      0.79       453
           1       0.75      0.67      0.71       260
           2       0.67      0.07      0.13        28
           3       0.00      0.00      0.00         2

    accuracy                           0.72       964
   macro avg       0.60      0.41      0.43       964
weighted avg       0.72      0.72      0.70       964

classification_report round
              precision    recall  f1-score   support

          -3       1.00      0.22      0.36         9
          -2       0.73      0.43      0.54       110
          -1       0.64      0.74      0.69       219
           0       0.51      0.59      0.55       181
           1       0.58      0.79      0.67       287
           2       0.88      0.25      0.39       154
           3       0.00

,Mean Absolute Error (MAE),Mean Squared Error (MSE),Root Mean Squared Error (RMSE),R-squared (R2),Mean Absolute Percentage Error (MAPE)
0,0.408618,0.314902,0.561162,0.803992,2.548211
